<center><font size="+4">Solving the diffusion approximation to the birth-immigration-death process</font></center>

# Forward Laplace transform

In [2]:
diffusioneqn (rho, D, s, M) := 
    diff (rho, tau) -
    D * diff (x * rho, x, 2) +
    s * diff (x * rho, x, 1) +
    M * diff (rho, x);

(%o2) diffusioneqn(rho, D, s, M) := diff(rho, tau) - D diff(x rho, x, 2)
                                         + s diff(x rho, x, 1) + M diff(rho, x)

In [3]:
cgfeqn (gam, D, s, M) := 
    diff (gam, tau) +
    (D * theta^2 - s* theta) * diff (gam, theta) -
    M * theta;

(%o3) cgfeqn(gam, D, s, M) := diff(gam, tau)
                                    2
                          + (D theta  - s theta) diff(gam, theta) + (- M) theta

# Characteristic equation

Compute the integral for the implicit solution.

In [4]:
logcontract(integrate(1/(D * thp^2 - s * thp), thp));

                                    D thp - s
                                log(---------)
                                       thp
(%o4)                           --------------
                                      s

Solve the implicit equation.

In [5]:
thetasol : solve ([((D* theta - s) * theta0) / ((D* theta0 - s) * theta) =
    %e^(s*u)], [theta])[1];
theta0sol : solve ([((D* theta - s) * theta0) / ((D* theta0 - s) * theta) =
    %e^(s*u)], [theta0])[1];

                                        s theta0
(%o5)              theta = - -------------------------------
                                              s u
                             (D theta0 - s) %e    - D theta0

                                               s u
                                     s theta %e
(%o6)                theta0 = ---------------------------
                                        s u
                              D theta %e    - D theta + s

Check the solution.

In [6]:
subst(0, u,  thetasol);

(%o7)                           theta = theta0

In [8]:
chareqn(theta) := diff(theta, u) - D * theta^2 + s * theta;

                                                       2
(%o9)        chareqn(theta) := diff(theta, u) - D theta  + s theta

In [9]:
factor(chareqn(part(thetasol, 2)));

(%o10)                                 0

# Homogeneous solution

Re-expressing $u$ in terms of $\tau$ and including the initial condition $P(x,0) = \delta(x-x_0) \Rightarrow \Gamma_{\mathrm{hom}}(\theta, 0)=x_0 \, \theta$, where $\Rightarrow$ indicates taking the logarithm of the Laplace transform, yields

In [10]:
gamhom : x0 * s * theta * %e^(s*tau) / 
    (D * (%e^(s*tau) - 1) * theta + s);

                                  s tau
                              s %e      theta x0
(%o11)                     -------------------------
                                s tau
                           D (%e      - 1) theta + s

In [11]:
factor(gamhom - x0 * subst(tau, u, part(theta0sol, 2)));

(%o12)                                 0

Noting $M=0$ in the homogeneous case

In [12]:
factor(cgfeqn (gamhom, D, s, 0));

(%o13)                                 0

# Inhomogeneous solutions

Find the characteristic which passes through a given point (theta_f, tau_f).

In [13]:
th0sol : solve([thetaf = 
        subst(uf, u, part(thetasol, 2))], [theta0])[1];

                                               s uf
                                    s thetaf %e
(%o14)              theta0 = ------------------------------
                                        s uf
                             D thetaf %e     - D thetaf + s

In [14]:
thfsol : factor(psubst(th0sol, thetasol));

                                              s uf
                                   s thetaf %e
(%o15)        theta = ------------------------------------------
                                 s uf              s u       s u
                      D thetaf %e     - D thetaf %e    + s %e

In [15]:
subst(uf, u, thfsol);

(%o16)                          theta = thetaf

Integrate along that characteristic.

In [16]:
ginhint : integrate((thetaf*s*%e^(uf*s))/
    (s*%e^(s*u)-D*thetaf*%e^(s*u)+D*thetaf*%e^(uf*s)), u);

                            - s uf
                  s uf  u %e
(%o17) s thetaf %e     (----------
                         D thetaf
                            - s uf                      s u              s uf
                          %e       log((D thetaf - s) %e    - D thetaf %e    )
                        - ----------------------------------------------------)
                                               D s thetaf

In [17]:
logcontract(ev(factor(subst(uf, u, ginhint) - subst(0, u, ginhint)), 
        logexpand=super));

                                     s uf
                          D thetaf %e     - D thetaf + s
                      log(------------------------------)
                                        s
(%o18)                -----------------------------------
                                       D

In [18]:
logcontract(ev(factor(subst(uf, u, ginhint) - subst(0, u, ginhint)), 
        logexpand=super));

                                     s uf
                          D thetaf %e     - D thetaf + s
                      log(------------------------------)
                                        s
(%o19)                -----------------------------------
                                       D

Adding back the factor of $M$ provides the solution to the inhomogeneous CGF.

In [19]:
gaminh : (M/D) * log( (D/s) * (%e^(-s * tau) - 1) * theta + 1);

                                 - s tau
                            D (%e        - 1) theta
                      M log(----------------------- + 1)
                                       s
(%o20)                ----------------------------------
                                      D

# Inverse Laplace transform

## Homogeneous solution

We proceed by constructing a change of variable to place the inverse Laplace transform integral into a form enabling the identification of a Bessel function solution. Combining the form of the Laplace transform with the form of `gamhom` we have

In [20]:
schemintgr : x * theta  - a * theta / (b * theta + 1);

                                         a theta
(%o21)                       theta x - -----------
                                       b theta + 1

The following change of variable $\theta \rightarrow \theta'$ will render `shemintgr` a function of $\theta' + \frac{1}{\theta'}$ as expected for a Bessel function

In [53]:
thetaChV: c * thp - 1/b;

                                           1
(%o59)                             c thp - -
                                           b

In [54]:
schemintgrthp: expand(subst(thetaChV, theta, schemintgr));

                                    x      a       a
(%o60)                    c thp x - - + -------- - -
                                    b    2         b
                                        b  c thp

Since we are looking for a form that includes a single factor of $\theta' + \frac{1}{\theta'}$ we look for $c x = \frac{a}{b^2 c}$

In [62]:
cbessel: a^(1/2)*x^(-1/2)/b$
thetaChVab: subst(cbessel, c, thetaChV);
schemintgrelimc: expand(subst(thetaChVab, theta, schemintgr));

                                sqrt(a) thp   1
(%o77)                          ----------- - -
                                 b sqrt(x)    b

                  x    sqrt(a) thp sqrt(x)   sqrt(a) sqrt(x)   a
(%o78)         (- -) + ------------------- + --------------- - -
                  b             b                 b thp        b

To provide $\theta$ in terms of $\theta'$ we substitute the values of $a$ and $b$ given by the homogeneous solution `gamhom`. First we note that if we define $a = x_0  e^{s \tau}$ and $b = \frac{D}{s} (e^{s \tau} -1)$

In [45]:
avalue: x0 * %e^(s*tau);
bvalue: (D/s) * (%e^(s*tau)- 1);

                                    s tau
(%o50)                            %e      x0

                                     s tau
                                D (%e      - 1)
(%o51)                          ---------------
                                       s

that `gamhom` $ = \frac {a \theta}{b \theta +1}$

In [46]:
factor(gamhom - avalue*theta/(bvalue*theta +1));

(%o52)                                 0

Now we substitute values of $a$ and $b$ into the terms of `schemintgrelimc`


In [59]:
factor(psubst([a = avalue, 
               b = bvalue],
               thetaChVab));

                           s tau
                           -----
                             2
                      s (%e      thp sqrt(x0) - sqrt(x))
(%o68)                ----------------------------------
                                s tau
                           D (%e      - 1) sqrt(x)

In [50]:
factor(psubst([a = avalue, 
               b = bvalue],
               -a/b - x/b));

                                    s tau
                               s (%e      x0 + x)
(%o56)                       - ------------------
                                     s tau
                                D (%e      - 1)

In [51]:
factor(psubst([a = avalue, 
               b = bvalue],
               sqrt(a*x)/b));

                                 s tau
                                 -----
                                   2
                             s %e      sqrt(x x0)
(%o57)                       --------------------
                                    s tau
                               D (%e      - 1)

## Inhomogeneous solution

In [26]:
invinhvarchange : theta = 
    thetap/x - s / (D * (%e^(s * tau) - 1));

                               thetap          s
(%o27)                 theta = ------ - ---------------
                                 x           s tau
                                        D (%e      - 1)

In [27]:
factor(psubst(invinhvarchange, 
  (D/s) * (%e^(s * tau) - 1) * theta + 1));

                                 s tau
                            D (%e      - 1) thetap
(%o28)                      ----------------------
                                     s x

In [28]:
inhdifsol : x^(M/D - 1) * 
    %e^(-s*x/(D * (%e^(s * tau) - 1))) /
    ((%e^(s * tau) - 1)^(M/D));

                                            s x
                                    - ---------------
                                           s tau
                          M/D - 1     D (%e      - 1)
                         x        %e
(%o29)                   ----------------------------
                                  s tau     M/D
                               (%e      - 1)

In [29]:
factor(diffusioneqn (inhdifsol, D, s, M));

(%o30)                                 0